In [ ]:
# !pip install IPython
from IPython.display import display, HTML, Image

In [ ]:
!git clone https://github.com/KU-DIC/LG_time_series_day16.git #코랩 사용

# [모델 평가 지표]
## 이상치 탐지 모델 평가 지표

##### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- 함수 ( ) 안에서 shift+tab: arguments description. shift+tab+tab은 길게 볼 수 있도록

## 1. 모듈 불러오기

In [ ]:
''' 데이터 전처리 패키지 '''
import numpy as np
import pandas as pd

''' 기계학습 모델 구축 및 평가 패키지 '''
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import LocalOutlierFactor
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, f1_score
from sklearn.model_selection  import train_test_split
from scipy.stats import gmean

''' 데이터 시각화 패키지 '''
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

''' 경고 숨기기 '''
import warnings
warnings.filterwarnings(action = 'ignore')

''' 한글 폰트 설정 '''
plt.rc('font', family = 'Malgun Gothic')

## 2. 데이터 불러오기: Boston Housing Data

#### https://archive.ics.uci.edu/ml/datasets.php

#### 데이터 구조  
- 데이터: 1978년 보스턴 주택 가격  
- 관측치 개수: 506개
- 변수 개수: 설명변수 13개 / 반응변수 1개

#### 설명 변수(원인: 예측값을 설명할 수 있는 변수) -> "X"
- CRIM: 범죄율  
- INDUS: 비소매상업지역 면적 비율  
- NOX: 일산화질소 농도  
- RM: 주택당 방 수  
- LSTAT: 인구 중 하위 계층 비율  
- B: 인구 중 흑인 비율  
- PTRATIO: 학생/교사 비율  
- ZN: 25,000 평방피트를 초과 거주지역 비율  
- CHAS: 찰스강의 경계에 위치한 경우는 1, 아니면 0  
- AGE: 1940년 이전에 건축된 주택의 비율  
- RAD: 방사형 고속도로까지의 거리  
- DIS: 직업센터의 거리  
- TAX: 재산세율  

#### 반응 변수(결과: 예측하고자 하는 값) -> "Y"
- MEDV: 주택가격

In [ ]:
# 데이터 불러오기
boston = load_boston()

# 데이터 설명
print(boston.DESCR)

In [ ]:
# 데이터 크기 및 사용 변수
print("데이터 크기 : ", boston.data.shape)
print("사용 변수 : ", boston.feature_names)

## 3. 데이터 전처리

$\mathbf{X}$: 설명변수/입력변수 <br>
$\mathbf{y}$: 반응변수/출력변수

- X, Y 데이터 프레임 만들기

In [ ]:
X = pd.DataFrame(boston.data, columns = boston.feature_names)
y = pd.DataFrame(boston.target, columns = ['MEDV'])

In [ ]:
# X 예시 - 상위 5개 샘플 출력
X.head()

In [ ]:
# Y 예시 - 상위 5개 샘플 출력
y.head()

- X, Y 데이터 프레임 합치기

In [ ]:
data = pd.concat([X, y], axis = 1)
data.head()

## 4. 탐색적 데이터 분석

- boxplot을 통해 각 데이터 별 outlier 확인 

In [ ]:
sns.boxplot(x = "variable", y = "value", data = pd.melt(data))

### 데이터 분포 확인 

- describe(): 다양한 통계량을 요약해서 제공하는 함수 


In [ ]:
data.describe()

- 4 분위수 기준으로 모든 변수에 대한 이상치 개수 확인 

In [ ]:
Image('/content/LG_time_series_day16/image/intro8.png')

In [ ]:
# 변수 별 outlier 개수, 비율 확인

def detect_outliers(df):
    cols = list(df)
    outliers = pd.DataFrame(columns = ['Feature', 'Number of Outliers'])
    for column in cols:
        if column in df.select_dtypes(include=np.number).columns:
            q1 = df[column].quantile(0.25)
            q3 = df[column].quantile(0.75)
            iqr = q3 - q1
            fence_low = q1 - (1.5*iqr)
            fence_high = q3 + (1.5*iqr)
            outliers = outliers.append({'Feature':column, 'Number of Outliers':df.loc[(df[column] < fence_low) | (df[column] > fence_high)].shape[0],'Proportion of Outliers':(df.loc[(df[column] < fence_low) | (df[column] > fence_high)].shape[0])/len(df)}, ignore_index=True)
    return outliers

detect_outliers(data)

## 5. 모델링

- LinearRegression: 선형회귀모형 
- LinearRegression(fit_intercept, normalize, copy_X, n_jobs)
  - fit_intercept : 모형에 상수항 (절편)이 있는가 없는가를 결정하는 인qk (default : True)
  - normalize : 매개변수 무시 여부
  - copy_X : X의 복사 여부
  - n_jobs : 계산에 사용할 작업 수


---
- Local Outlier Factor: 이상치 탐지 모델 
  - 관측치의 주변 데이터를 이용해 국소적 관점으로 이상치 정도를 파악하는 방법론 



- 선형회귀모형

In [ ]:
y = data['MEDV']
X = data.iloc[:,0:13]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# 선형회귀모형 훈련 
lm = LinearRegression()
lm.fit(X_train, y_train)

- Local Outlier Factor

In [ ]:
# LOF를 통한 결과를 바탕으로 outlier 제거한 새로운 데이터 생성 
lof = LocalOutlierFactor()
yhat = lof.fit_predict(X_train) # yhat이 -1일 경우 outlier 
mask = yhat != -1 
X_train_lof, y_train_lof = X_train.iloc[mask, :], y_train.iloc[mask]
print(X_train_lof.shape, y_train_lof.shape)

In [ ]:
lm_lof = LinearRegression()
lm_lof.fit(X_train_lof, y_train_lof)
pred_lof = lm_lof.predict(X_test)

- 로지스틱회귀모형

In [ ]:
# label 생성
data['label'] = np.where(data["MEDV"] > np.mean(y_train), 1, 0)

In [ ]:
y = data['label']
X = data.iloc[:,0:13]

X_train_logit, X_test_logit, y_train_logit, y_test_logit = train_test_split(X, y, test_size = 0.3, random_state = 0)

print(X_train_logit.shape, X_test_logit.shape, y_train_logit.shape, y_test_logit.shape)

In [ ]:
# 로지스틱회귀모형 
logit = LogisticRegression()
logit.fit(X_train_logit, y_train_logit)

## 6. 모델 해석

- coef (회귀계수): X가 한단위 증가할 때 Y의 변화량
- Intercept (절편): 독립변수가 모두 0일때 종속변수 값 

In [ ]:
# 선형회귀모형 
print('labels\n',X.columns)
print("="*50)
print('Coefficients: \n', lm.coef_)
print("="*50)
print('Intercept: \n', lm.intercept_)

In [ ]:
# 이상치 탐지 선형회귀모형 
print('labels\n',X.columns)
print("="*50)
print('Coefficients: \n', lm_lof.coef_)
print("="*50)
print('Intercept: \n', lm_lof.intercept_)

## 7. 모델 성능 평가

In [ ]:
data['MEDV_REG'] = lm.predict(data.iloc[:,0:13])
data[['MEDV', 'MEDV_REG']].head()

In [ ]:
# 실제값과 예측값 분포 시각화 
plt.scatter(data['MEDV'], data['MEDV_REG'], s = 5 )
plt.xlabel("Prices")
plt.ylabel("Predicted Prices")
plt.title("Real vs Predicted Housing Prices")

In [ ]:
# 0을 기준으로 잔차 시각화 
plt.scatter(lm.predict(X_train), lm.predict(X_train) - y_train, c = 'b', s = 30, alpha = 0.4)
plt.scatter(lm.predict(X_test), lm.predict(X_test) - y_test, c ='g', s = 30 )

plt.hlines(y = 0, xmin = -5, xmax = 55)
plt.title("Residuals")
plt.ylabel("Residuals")

### 다양한 회귀 모델 성능 평가 지표들 (예측 모델, 분류 모델)

#### Mean Squared Error (평균 제곱 오차): $\frac{1}{n} \sum_{i=1}^{n} (y_{i} - \hat{y}_{i})^{2}$

####  Mean Absolute Error (평균 절대 오차): $\frac{1}{n} \sum_{i=1}^{n} |\hat{y}_{i}- y_{i}|$

####  Geometric Mean (기하 평균): $\left(\prod _{i=1}^{n}x_{i}\right)^{\frac {1}{n}}={\sqrt[{n}]{x_{1}x_{2}\cdots x_{n}}}$

####  F1 socre: $\frac{2*Precision*Recall}{Precision+Recall} = \frac{2*TP}{2*TP+FP+FN}$

### **선형회귀모형 평가** 



In [ ]:
true = data['MEDV']
pred = data['MEDV_REG']

In [ ]:
# 예측값과 실제값 차이 계산 
error = true - pred

In [ ]:
# Gmean 함수 정의 
def geo_mean(iterable):
    a = np.array(iterable)
    return a.prod()**(1.0/len(a))

In [ ]:
# mse, mae, gmean 계산 
MSE_value = mean_squared_error(true, pred)
MAE_value = mean_absolute_error(true, pred)
GMEAN_value = geo_mean(error)

print("Training MSE : {:.3f}".format(MSE_value))
print("Training MAE : {:.3f}".format(MAE_value))
print("Training GMean : {:.3f}".format(GMEAN_value))

### **이상치 탐지 선형회귀모형 평가** 

In [ ]:
data['MEDV_REG_LOF'] = lm_lof.predict(data.iloc[:,0:13])

In [ ]:
true_lof = data['MEDV']
pred_lof = data['MEDV_REG_LOF']

In [ ]:
# 예측값과 실제값 차이 계산 
error = true_lof - pred_lof

In [ ]:
# mse, mae, gmean 계산 
MSE_value = mean_squared_error(true_lof, pred_lof)
MAE_value = mean_absolute_error(true_lof, pred_lof)
GMEAN_value = geo_mean(error)

print("Training MSE : {:.3f}".format(MSE_value))
print("Training MAE : {:.3f}".format(MAE_value))
print("Training GMean : {:.3f}".format(GMEAN_value))

### **로지스틱 회귀모형 평가** 

In [ ]:
# F1 score 계산

data['label_REG'] = logit.predict(data.iloc[:,0:13])

y_true = data['label']
y_pred = data['label_REG']

In [ ]:
F1_value = f1_score(y_true, y_pred)
print("Training F1 score : {:.3f}".format(F1_value))

---